### 하나은행 예금 데이터 수집
- requests, bs4 패키지를 사용하여 하나은행 예금 데이터를 아래와 같이 수집하세요.
    - 상품에 대한 금리는 최고 금리가 출력되도록 하세요.
    - items_df : 상품 리스트에 대한 데이터 프레임
    - items_detail_df : 상품에 대한 상세 정보를 가지는 데이터 프레임
- URL : https://www.kebhana.com/cont/mall/mall08/mall0805/index.jsp?_menuNo=62608

In [1]:
import requests
from bs4 import BeautifulSoup

#### 1. requests 로 html 문자열 가져와서 parsing

In [1]:
url = "https://www.kebhana.com/cont/mall/mall08/mall0805/index.jsp?_menuNo=62608"
response = requests.post(url)
response

<Response [200]>

In [7]:
dom = BeautifulSoup(response.content, "html.parser")

# chrome browser selector 사용
elements = dom.select('#contents > div.banking-content > div.banking-row-area > div.wrap-product-list > ul > li')

len(elements)

15

#### 2. 상품 상세 데이터 가져오기 1

In [47]:
element = elements[1]

In [48]:
# product-title
data1 = element.select_one(".product-tit > i").text

# badge
data2 = element.select_one(".product-tit > .rainbow-color").text.strip()\
.replace("\r", "").replace("\n", "").replace("\r", "").replace("\t", "").replace("\xa0", " ")

# title
data3 = element.select_one(".product-tit > em > a").text

# title-desc
data4 = element.select_one(".product-tit > .tit-desc").text

# link
data5 = "https://www.kebhana.com" +  element.select_one(".product-tit > em > a").get("href")

data1, data2, data3, data4, data5

('정기예금',
 '베스트 인터넷 스마트폰',
 '하나머니세상 정기예금',
 '하나멤버스 회원손님이 예금의 이자를 하나머니로 적립동의하면 우대금리 제공! 하나카드사의 1Q카드 결제실적까지 보유하면 예금이자의 세금만큼 하나머니로 돌려드리는 정기예금 상품입니다.',
 'https://www.kebhana.com/cont/mall/mall08/mall0801/mall080101/1445172_115126.jsp')

#### 3. 상품 상세 데이터 가져오기 2

In [50]:
link = data5
print(link)

https://www.kebhana.com/cont/mall/mall08/mall0801/mall080101/1445172_115126.jsp


In [51]:
response = requests.get(link)
response

<Response [200]>

In [52]:
dom = BeautifulSoup(response.content, "html.parser")
title_elements = dom.select(".prodcutInfo > dt") 
desc_elements = dom.select(".prodcutInfo > dd")

len(title_elements), len(desc_elements)

(14, 14)

In [53]:
title_elements[0].text, desc_elements[0].text

('상품특징',
 '하나멤버스 회원손님이 예금의 이자를 하나머니로 적립동의하면 우대금리 제공! 하나카드사의 1Q카드 결제실적까지 보유하면 예금이자의 세금만큼 하나머니로 돌려드리는 정기예금 상품입니다.')

In [54]:
# 금리 데이터가 없고 자바스크립트 코드가 있음 : iframe에서 데이터를 가져와서 보여줌
title_elements[6].text, desc_elements[6].text.strip()

('금리', '금리안내')

In [55]:
# iframe url을 가져와서 request
i_url = "https://www.kebhana.com" + desc_elements[6].select_one("#zff").get("src")
print(i_url)

https://www.kebhana.com/app/portal/mkt/contents/rate_p02_01.do?actionCode=02&subMenu=03&RateIrdCd=0703000200019&pageYn=Y


In [56]:
response = requests.get(i_url)
response

<Response [200]>

In [57]:
# 정규표현식으로 이자율 데이터를 모두 가져옴
numbers = [float(data) for data in re.findall(">([0-9]{1,2}\.?[0-9]{0,2})%", response.text)]
numbers

[0.2, 0.4]

In [59]:
# 최소값, 최대값
np.min(numbers), np.max(numbers)

(0.2, 0.4)

#### 4. 함수로 만들기

In [60]:
# 예금 데이터 수집 함수
def get_list():
    url = "https://www.kebhana.com/cont/mall/mall08/mall0805/index.jsp?_menuNo=62608"
    params = {
        "catId": "spb_2811",
        "startCount": "0",
        "sort": "search_order/DESC",
        "collection": "pro_product",
        "range": "A",
        "searchField": "ALL",
        "tabStatus": "1",    
    }

    response = requests.post(url)
    dom = BeautifulSoup(response.content, "html.parser")
    
    elements = dom.select('#contents > div.banking-content > div.banking-row-area > div.wrap-product-list > ul > li')

    datas = []
    for element in elements:
        datas.append({
            "product-title": element.select_one(".product-tit > i"),
            "badge": element.select_one(".product-tit > .rainbow-color").text.strip().replace("\r", "").replace("\n", "").replace("\r", "").replace("\t", ""),
            "title": element.select_one(".product-tit > em > a").text,
            "title-desc": element.select_one(".product-tit > .tit-desc").text,
            "link": "https://www.kebhana.com" +  element.select_one(".product-tit > em > a").get("href"),
        })
        
    return pd.DataFrame(datas)

In [61]:
def detail_datas(d_link, idx):
    response = requests.get(d_link)
    dom = BeautifulSoup(response.content, "html.parser")
    title_elements = dom.select(".prodcutInfo > dt") 
    desc_elements = dom.select(".prodcutInfo > dd")
    
    datas = []
    for title, desc in zip(title_elements, desc_elements):

        title_txt = title.text
        
        # 제목이 금리이면 iframe url로 데이터 가져옴
        if title_txt == "금리":
            i_url = "https://www.kebhana.com" + desc.select_one("#zff").get("src")
            response = requests.get(i_url)
            desc_txt = BeautifulSoup(response.text, "html.parser").text.strip().replace("\n", "")
            numbers = [float(data) for data in re.findall("([0-9]{1,2}\.?[0-9]{0,2})%", desc_txt)]
            desc_txt = np.max(numbers)
            
        # 그외에는 그냥 text 데이터 가져옴    
        else:
            desc_txt = desc.text.strip().replace("\n", "").replace("\r", "").replace("\t", "").replace("\xa0", " ")
            
        datas.append({
            "product_idx": idx,     # 상품 index 값
            "title_txt": title_txt, # 상품명
            "desc_txt": desc_txt,   # 최고 금리
        })
        
    return pd.DataFrame(datas)

#### 5. 함수실행

In [62]:
# 상세 데이터 가져오기 1
items_df = get_list()
items_df.tail(2)

,product-title,badge,title,title-desc,link
13,[정기예금],,표지어음,은행이 보유하고 있는 원어음(상업어음 또는 무역어음)을 근거로 발행하는 배서양도가 ...,https://www.kebhana.com/cont/mall/mall08/mall0...
14,[정기예금],,1년 연동형 정기예금,서울보증보험의 보증서 발급 담보용 정기예금,https://www.kebhana.com/cont/mall/mall08/mall0...


In [64]:
# 상세 데이터 가져오기 2
dfs = []
for idx, data in items_df.iterrows():
    print(idx, end=" ")
    dfs.append(detail_datas(data.link, idx))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

In [65]:
# 데이터 프레임 합치기
item_details_df = pd.concat(dfs)
item_details_df.reset_index(drop=True, inplace=True)

In [67]:
items_df.head(2)

,product-title,badge,title,title-desc,link
0,[정기예금],베스트 신상품 스마트폰,하나 원큐 정기예금,간편하게 우대금리를 제공하는 스마트폰 예금,https://www.kebhana.com/cont/mall/mall08/mall0...
1,[정기예금],베스트 인터넷 스마트폰,하나머니세상 정기예금,하나멤버스 회원손님이 예금의 이자를 하나머니로 적립동의하면 우대금리 제공! 하나카드...,https://www.kebhana.com/cont/mall/mall08/mall0...


In [68]:
item_details_df[:20]

,product_idx,title_txt,desc_txt
0,0,상품특징,(스마트폰 전용 예금) 마케팅동의와 하나오픈뱅킹 서비스를 통하여 상품 신규 시 우대...
1,0,가입대상,실명의 개인 및 개인사업자
2,0,예금종류,정기예금
3,0,가입기간,1년
4,0,가입금액,"1백만원 이상 ~ 3천만원 이내 (단, 오픈뱅킹 1일 이체한도는 1천만원)"
5,0,이자지급방법,만기일시지급식
6,0,금리,0.8
7,0,우대금리,"최대 연 0.2% (2019.12.18 기준, 세전)이 예금의 가입자가 아래 항목을..."
8,0,특별금리,이 예금 가입시 하나오픈뱅킹서비스 출금이체(다른은행 계좌)로 신규한 경우 0.2%추...
9,0,상품내용변경에 관한사항,변경사항상품내용 변경에 관한 사항 (2020.07.01 변경) <중도해지금리변경...


In [69]:
# 상품별 최고 금리
item_details_df[item_details_df["title_txt"] == "금리"]

,product_idx,title_txt,desc_txt
6,0,금리,0.8
20,1,금리,0.4
34,2,금리,0.75
48,3,금리,0.75
60,4,금리,0.5
73,5,금리,0.65
85,6,금리,1
98,7,금리,0.75
110,8,금리,0.6
122,9,금리,0.85
